In [1]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
from utils import Dynamic_dataGen

key = os.environ.get("OPEN_AI_KEY")
# obj = utils.Static_dataGen(key)

In [32]:
f = open('./tool_list.txt', 'r').read()

In [33]:
f

'def works_list(\n    applies_to_part=None, created_by=None, issue_priority=None,\n    issue_rev_orgs=None, limit=50, owned_by=None, stage_name=None,\n    ticket_needs_response=False, ticket_rev_org=None,\n    ticket_severity=None, ticket_source_channel=None, work_type=None\n):\n    """\n    Returns a list of work items matching the request\n\n    Parameters:\n        applies_to_part (list): Filters for work belonging to any of the provided parts\n        created_by (list): Filters for work created by any of these users\n        issue.priority (list): Filters for issues with any of the provided priorities. Allowed values: p0, p1, p2, p3\n        issue.rev_orgs (list): Filters for issues with any of the provided Rev organizations\n        limit (int): The maximum number of works to return. The default is \'50\'\n        owned_by (list): Filters for work owned by any of these users\n        stage.name (list): Filters for records in the provided stage(s) by name\n        ticket.needs_resp